In [ ]:
#
# This file contains the common functions and definitions used in the Egeria Hands on
# Lab Notebooks.
#

# These functions define the location and names of Coco Pharmaceutical's
# Open Metadata and Governance (OMAG) Server Platforms where the metadata servers
# and governance servers run.  The `os.environ.get` function tests for the presence
# of the environment variables that define the platform network addresses in the
# Docker and Kubernetes runtime environments.  If the environment variables are not
# present then the localhost defaults are used.

import os

corePlatformURL      = os.environ.get('corePlatformURL', 'http://localhost:8080')
corePlatformName     = "Core Platform"

dataLakePlatformURL  = os.environ.get('dataLakePlatformURL', 'http://localhost:8081')
dataLakePlatformName = "Data Lake Platform"

devPlatformURL       = os.environ.get('devPlatformURL', 'http://localhost:8082')
devPlatformName      = "Dev Platform"


# The OMAG Server Platforms host one to many OMAG Servers.  An OMAG Server could be
# a metadata server or a specialized governance server.  Its behavior is determined
# by a configuration document that defines which OMAG services are activated.
# All OMAG Server Platforms support the administration commands to define a server's
# configuration document.  It is also possible to create configuration documents
# through admin calls to one OMAG Server Platform and then deploy them to the
# OMAG Server Platform where they are to run.  In the Egeria hands on lab, the
# OMAG Server configuration is created on the dev platform and deployed to the
# production platforms as needed.

adminPlatformURL = devPlatformURL


# Gary Geeke is the IT Administration Lead at Coco Pharmaceuticals.
# He does all of the configuration for the OMAG Servers.

adminUserId = "garygeeke"


# These are the names of the metadata servers used by Coco Pharmaceuticals.  Each metadata
# server runs as an OMAG Server on one of the OMAG Server Platforms

cocoMDS1PlatformURL = dataLakePlatformURL
cocoMDS1Name        = "cocoMDS1"

cocoMDS2PlatformURL = corePlatformURL
cocoMDS2Name        = "cocoMDS2"

cocoMDS3PlatformURL = corePlatformURL
cocoMDS3Name        = "cocoMDS3"

cocoMDS4PlatformURL = dataLakePlatformURL
cocoMDS4Name        = "cocoMDS4"

cocoMDS5PlatformURL = corePlatformURL
cocoMDS5Name        = "cocoMDS5"

cocoMDS6PlatformURL = corePlatformURL
cocoMDS6Name        = "cocoMDS6"

cocoMDSXPlatformURL = devPlatformURL
cocoMDSXName        = "cocoMDSx"


# These are the names of the governance servers used in Coco Pharmaceuticals' data lake.
# Each governance server runs as an OMAG Server on the Data Lake OMAG Server Platform.
# They also connect to a metadata server to retrieve their configuration and store their
# results.

findItDL01PlatformURL = devPlatformURL
findItDL01Name        = "findItDL01"
findItDL01ServerType  = "Discovery Server"
findItDL01MDS         = "cocoMDS1"

fixItDL01PlatformURL  = devPlatformURL
fixItDL01Name         = "fixItDL01"
fixItDL01ServerType   = "Stewardship Server"
fixItDL01MDS          = "cocoMDS1"


#
# Common processing of REST API errors.
#

import requests
import pprint
import json

def printResponse(response):
    prettyResponse = json.dumps(response.json(), indent=4)
    print(" ")
    print("Response: ")
    print(prettyResponse)
    print(" ")
    
def printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response):
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            print("Unexpected response from server " + serverName)
            printResponse(response)
        else:
            exceptionErrorMessage = response.json().get('exceptionErrorMessage')
            exceptionSystemAction = response.json().get('exceptionSystemAction')
            exceptionUserAction   = response.json().get('exceptionUserAction')
            if exceptionErrorMessage != None:
                print(exceptionErrorMessage)
                print(" * " + exceptionSystemAction)
                print(" * " + exceptionUserAction)
            else:
                print("Unexpected response from server " + serverName)
                printResponse(response)
    else:
        print("Unexpected response from server platform " + serverPlatformName + " at " + serverPlatformURL)
        printResponse(response)


#
# Metadata server configuration functions.  These functions add definitions
# to an OMAG server's configuration document
#

def postAndPrintResult(url, json=None, headers=None):
    print("   ...... (POST", url, ")")
    response = requests.post(url, json=json, headers=headers)
    print("   ...... Response: ", response.json())

def configurePlatformURL(serverName, serverPlatform):
    print("   ... configuring the platform the server will run on...")
    url = adminCommandURLRoot + serverName + '/server-url-root?url=' + serverPlatform
    postAndPrintResult(url)

def configureMaxPageSize(serverName, maxPageSize):
    print ("   ... configuring the maximum page size...")
    url = adminCommandURLRoot + serverName + '/max-page-size?limit=' + maxPageSize
    postAndPrintResult(url)

def configureServerType(serverName, serverType):
    print ("   ... configuring the server's type...")
    url = adminCommandURLRoot + serverName + '/server-type?typeName=' + serverType
    postAndPrintResult(url)

def configureOwningOrganization(serverName, organizationName):
    print ("   ... configuring the server's owning organization...")
    url = adminCommandURLRoot + serverName + '/organization-name?name=' + organizationName
    postAndPrintResult(url)

def configureUserId(serverName, userId):
    print ("   ... configuring the server's userId...")
    url = adminCommandURLRoot + serverName + '/server-user-id?id=' + userId
    postAndPrintResult(url)

def configurePassword(serverName, password):
    print ("   ... configuring the server's password (optional)...")
    url = adminCommandURLRoot + serverName + '/server-user-password?password=' + password
    postAndPrintResult(url)

def configureSecurityConnection(serverName, securityBody):
    print ("   ... configuring the server's security connection...")
    url = adminCommandURLRoot + serverName + '/security/connection'
    postAndPrintResult(url, json=securityBody, headers=jsonContentHeader)

def configureDefaultAuditLog(serverName):
    print ("   ... configuring the default audit log...")
    url = adminCommandURLRoot + serverName + '/audit-log-destinations/default'
    postAndPrintResult(url)
    
def configureMetadataRepository(serverName, repositoryType):
    print ("   ... configuring the metadata repository...")
    url = adminCommandURLRoot + serverName + '/local-repository/mode/' + repositoryType
    postAndPrintResult(url)

def configureDescriptiveName(serverName, collectionName):
    print ("   ... configuring the short descriptive name of the metadata stored in this server...")
    url = adminCommandURLRoot + serverName + '/local-repository/metadata-collection-name/' + collectionName
    postAndPrintResult(url)

def configureEventBus(serverName, busBody):
    print ("   ... configuring the event bus for this server...")
    url = adminCommandURLRoot + serverName + '/event-bus'
    postAndPrintResult(url, json=busBody, headers=jsonContentHeader)

def configureCohortMembership(serverName, cohortName):
    print ("   ... configuring the membership of the cohort...")
    url = adminCommandURLRoot + serverName + '/cohorts/' + cohortName
    postAndPrintResult(url)

def configureAccessService(serverName, accessService, accessServiceOptions):
    print ("   ... configuring the " + accessService + " access service for this server...")
    url = adminCommandURLRoot + serverName + '/access-services/' + accessService
    postAndPrintResult(url, json=accessServiceOptions, headers=jsonContentHeader)
    

# The OMAG Server Platform is a single executable (application) that can be started
# from the command line or a script or as part of a pre-built container environment
# such as docker-compose or kubernetes.  The function below checks that a specific
# server platform is running.

def checkServerPlatform(serverPlatformName, serverPlatformURL):
    try:
        isPlatformActiveURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/origin"
        response = requests.get(isPlatformActiveURL)
        if response.status_code == 200:
            print("   ", serverPlatformName, "is active")
            return True
        else:
            print("   ", serverPlatformName, "is not an OMAG Platform")
            return False
    except:
        print("   ", serverPlatformName, "is down - start it before proceeding")
        return False


# The OMAG Server Platform has the implementation of the open metadata and governance (OMAG) services.  These are
# divided into three groups: Common Services, Access Services and Governance Services.

def printServiceDescriptions(serverPlatformName, serviceGroupName, services):
    print(serviceGroupName, "for", serverPlatformName)
    for x in range(len(services)):
        serviceName = services[x].get('serviceName')
        serviceDescription = services[x].get('serviceDescription')
        serviceWiki = services[x].get('serviceWiki')    
        print (" * ", serviceName)
        print ("     ", serviceDescription)
        print ("     ", serviceWiki)
    print (" ")
    
def getServerPlatformServices(serverPlatformName, serverPlatformURL):
    getOMAGServicesURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/registered-services"
    response = requests.get(getOMAGServicesURL + "/common-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Common services", response.json().get('services'))
    response = requests.get(getOMAGServicesURL + "/access-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Access services", response.json().get('services'))
    response = requests.get(getOMAGServicesURL + "/governance-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Governance services", response.json().get('services'))


In [ ]:
# Each server is configured to define which services should be actived.  This configuration results in
# the creation of a configuration document.  This document is read when the server is started and
# drives the initialization of the services.

def checkServerConfigured(serverName, serverPlatformName, serverPlatformURL):
    isServerKnownOnPlatform = serverPlatformURL + "/open-metadata/admin-services/users/" + adminUserId + "/servers/" + serverName + "/configuration"
    response = requests.get(isServerKnownOnPlatform)
    if response.status_code == 200:
        serverConfig=response.json().get('omagserverConfig')
        auditTrail = serverConfig.get('auditTrail')
        if auditTrail is not None:
            print("           ...", serverName, "is configured")
            return True
        else:
            print("           ...", serverName, "needs configuring")
    else:
        print("   ...", serverPlatformName, "at", serverPlatformURL, "is down - unable to check server configuration")
        return False


# The OMAG Server Platform supports a call to return if a server is active.

def checkServerActive(serverName, serverPlatformName, serverPlatformURL):
    isServerActiveOnPlatform = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/servers/" + serverName + "/status"
    response = requests.get(isServerActiveOnPlatform)
    if response.status_code == 200:
        serverStatus = response.json().get('active')
        if serverStatus == True:
            print("           ...", serverName, "is active - ready to begin")
        else:
            print("           ...", serverName, "is down - needs to be started")
        return serverStatus
    else:
        print("   ...", serverPlatformName, "at", serverPlatformURL, "is down - unable to check server configuration")
        return False

    
# This is the call to start a server on a specific platform.  Once the server is running it is possible to
# make use of the open metadata and governance services that have been activated in the server.

def activateServerOnPlatform(serverName, serverPlatformName, serverPlatformURL):
    print ("                Starting server " + serverName + " ...")
    activateServerURL = serverPlatformURL + "/open-metadata/admin-services/users/" + adminUserId + '/servers/' + serverName + "/instance"
    response = requests.post(activateServerURL)
    if response.status_code == 200:
        return True
    else:
        print ("                   ..." + serverName + " failed to start")
        return False


# Once a server is active, it is possible to query the services that are active.

def getServerServices(serverName, serverPlatformName, serverPlatformURL):
    getServerActiveServicesURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/servers/" + serverName + "/services"
    print ("Services for server:", serverName)
    response = requests.get(getServerActiveServicesURL)
    if response.status_code == 200:
        serviceList = response.json().get('serverServicesList')
        for x in range(len(serviceList)):
            print (" * ", serviceList[x])
    else:
        print (response)

# This function checks whether a server is active and starts it if it is down.

def activateServerIfDown(serverName, serverPlatformName, serverPlatformURL):
    print("        Checking OMAG Server " + serverName)
    configured = checkServerConfigured(serverName, serverPlatformName, serverPlatformURL)
    if configured == True:
        active = checkServerActive(serverName, serverPlatformName, serverPlatformURL)
        if active == False:
            activateServerOnPlatform(serverName, serverPlatformName, serverPlatformURL)

# This function checks the platform is running and ensures the servers are started on it.
# It requests user action if either the platform is not running or the servers are not configured.
# Otherwise it should return with all of the servers running.

def activatePlatform(serverPlatformName, serverPlatformURL, hostedServerNames):
    available = checkServerPlatform(serverPlatformName, serverPlatformURL)
    if available == True:
        for x in range(len(hostedServerNames)):
            activateServerIfDown(hostedServerNames[x], serverPlatformName, serverPlatformURL)
   

In [ ]:
# The open metadata servers are linked together through the following open metadata cohorts.
# The servers linked via a cohort can exchange open metadata either through federated
# queries or metadata replication.

cocoCohort = "cocoCohort"
devCohort  = "devCohort"
iotCohort  = "iotCohort"
ctsCohort  = "ctsCohort"

def queryServerCohorts(serverName, serverPlatformName, serverPlatformURL):
    cohortNames = []
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohort-descriptions'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohorts=response.json().get('cohorts')
                if cohorts != None:
                    for x in range(len(cohorts)):
                        cohortName = cohorts[x].get('cohortName')
                        cohortNames.append(cohortName)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except:
        print("Platform " + serverPlatformURL + " is unavailable")
    return cohortNames


def printServerCohortsStatus(serverName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohort-descriptions'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohorts=response.json().get('cohorts')
                if cohorts == None:
                    print("Server " + serverName + " is not connected to any cohorts")
                else:
                    print("Server " + serverName + " is connected to the following cohorts:")
                    for x in range(len(cohorts)):
                        cohortName = cohorts[x].get('cohortName')
                        connectionStatus = cohorts[x].get('connectionStatus')
                        print (" * " + cohortName + " [" + connectionStatus + "]")
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except:
        print("Platform " + serverPlatformURL + " is unavailable")


def printCohortMember(cohortMember, localRegistration):
    serverName = cohortMember.get('serverName')
    serverType = cohortMember.get('serverType')
    metadataCollectionId = cohortMember.get('metadataCollectionId')
    metadataCollectionName = cohortMember.get('metadataCollectionName')
    registrationTime = cohortMember.get('registrationTime')
    if localRegistration == True:
        print("Registration details for local " + serverType + " " + serverName)
    else:    
        print("Registration details for remote " + serverType + " " + serverName)
    if (metadataCollectionId != None):
        print(" * Metadata collection id:   " + metadataCollectionId)
    if (metadataCollectionName != None):
        print(" * Metadata collection name: " + metadataCollectionName)
    if (registrationTime != None):
        print(" * Registration time:        " + registrationTime)
    repositoryConnection = cohortMember.get('repositoryConnection')
    if repositoryConnection != None:
        endpoint = repositoryConnection.get('endpoint')
        if endpoint != None:
            address = endpoint.get('address')
            if address != None:
                print(" * URL for metadata queries: " + address)
            else:
                print(" * URL for metadata queries: null")
        else:
            print(" * URL for metadata queries: no endpoint")
    else:
        print(" * URL for metadata queries: not supported")


def printLocalRegistration(serverName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/local-registration'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohortMember = response.json().get('cohortMember')
                printCohortMember(cohortMember, True)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except:
        print("Platform " + serverPlatformURL + " is unavailable")


def printLocalRegistrationForCohort(serverName, cohortName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohorts/' + cohortName + '/local-registration'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohortMember = response.json().get('cohortMember')
                printCohortMember(cohortMember, True)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except:
        print("Platform " + serverPlatformURL + " is unavailable")


def printRemoteRegistrations(serverName, cohortName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore = '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohorts/' + cohortName + '/remote-members'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohortMembers = response.json().get('cohortMembers')
                if cohortMembers != None:
                    for x in range(len(cohortMembers)):
                        printCohortMember(cohortMembers[x], False)
                else:
                    print("No remote members")
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except:
        print("Platform " + serverPlatformURL + " is unavailable")
    

def printServerCohorts(serverName, serverPlatformName, serverPlatformURL):
    print("Reviewing the cohort registry for server: " + serverName)
    print(" ")
    printLocalRegistration(serverName, serverPlatformName, serverPlatformURL)
    print(" ")
    cohorts = queryServerCohorts(serverName, serverPlatformName, serverPlatformURL)
    for x in range(len(cohorts)):
        print("Cohort " + cohorts[x] + " member details")
        printLocalRegistrationForCohort(serverName, cohorts[x], serverPlatformName, serverPlatformURL)
        printRemoteRegistrations(serverName, cohorts[x], serverPlatformName, serverPlatformURL)
        print(" ")
        

In [ ]:
# Perform basic checks to ensure the calling notebook has a good environment to work against.

print("\nChecking OMAG Server Platform availability...")

activatePlatform("Core Platform", corePlatformURL, ["cocoMDS2", "cocoMDS3", "cocoMDS5", "cocoMDS6"])
activatePlatform("Data Lake Platform", dataLakePlatformURL, ["cocoMDS1", "cocoMDS4", "discoDL01"])
activatePlatform("Dev Platform", devPlatformURL, ["cocoMDSx"])

print ("Done.")
print (" ")

# getServerPlatformServices("Dev Platform", devPlatformURL)
# print (" ")
# getServerServices("cocoMDS1", "Data Lake Platform", dataLakePlatformURL)
# print (" ")
# getServerServices("cocoMDS4", "Data Lake Platform", dataLakePlatformURL)

serverName = "cocoMDS2"
serverPlatformURL = 'http://localhost:8080'
serverPlatformName = "platformName"

print(" ")
printLocalRegistration(serverName, serverPlatformName, serverPlatformURL)
print(" ")
printServerCohortsStatus(serverName, serverPlatformName, serverPlatformURL)
print(" ")
printLocalRegistrationForCohort(serverName, "cocoCohort", serverPlatformName, serverPlatformURL)
print(" ")
printRemoteRegistrations(serverName, "cocoCohort", serverPlatformName, serverPlatformURL)
print(" ")

print(" ")
printServerCohorts(serverName, serverPlatformName, serverPlatformURL)


In [ ]:
activateServerOnPlatform("discoDL01", "Data Lake Platform", dataLakePlatformURL)